# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Checkpoint" data-toc-modified-id="Checkpoint-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Checkpoint</a></div><div class="lev1 toc-item"><a href="#Build-Model" data-toc-modified-id="Build-Model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Build Model</a></div><div class="lev2 toc-item"><a href="#Set-Hyperparameters" data-toc-modified-id="Set-Hyperparameters-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Set Hyperparameters</a></div><div class="lev2 toc-item"><a href="#Import-Libraries" data-toc-modified-id="Import-Libraries-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Import Libraries</a></div><div class="lev2 toc-item"><a href="#Build-Graph" data-toc-modified-id="Build-Graph-23"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Build Graph</a></div><div class="lev2 toc-item"><a href="#Model-Visualization" data-toc-modified-id="Model-Visualization-24"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Model Visualization</a></div><div class="lev2 toc-item"><a href="#Train" data-toc-modified-id="Train-25"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Train</a></div><div class="lev1 toc-item"><a href="#Evaluate" data-toc-modified-id="Evaluate-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Evaluate</a></div><div class="lev2 toc-item"><a href="#Naive-Evaluate" data-toc-modified-id="Naive-Evaluate-31"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Naive Evaluate</a></div><div class="lev2 toc-item"><a href="#Greedy-1-best-Search-Evaluate" data-toc-modified-id="Greedy-1-best-Search-Evaluate-32"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Greedy 1-best Search Evaluate</a></div><div class="lev2 toc-item"><a href="#Beam-Search-Evaluate" data-toc-modified-id="Beam-Search-Evaluate-33"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Beam Search Evaluate</a></div>

# Checkpoint

In [1]:
import h5py
import pickle

In [2]:
with h5py.File('allData.h5', 'r') as fh:
    x_train = fh['x_train'][:]
    tf_train = fh['tf_train'][:]
    y_train = fh['y_train'][:]
    x_val = fh['x_val'][:]
    tf_val = fh['tf_val'][:]
    y_val = fh['y_val'][:]
    x_train_all = fh['x_train_all'][:]
    tf_train_all = fh['tf_train_all'][:]
    y_train_all = fh['y_train_all'][:]
    x_test = fh['x_test'][:]
    tf_test = fh['tf_test'][:]
    y_test = fh['y_test'][:]
    embedding = fh['embedding'][:]

In [3]:
with open('index.pkl', 'rb') as fp:
    word2index, index2word = pickle.load(fp)

# Build Model

##  Set Hyperparameters

In [4]:
MAX_SENT_LEN = 64
MAX_ADJL_LEN = 3
VOCAB_SIZE = len(word2index)+1
NUM_CLASSES = VOCAB_SIZE
EMBEDDING_SIZE = 300
TF_EMBEDDING_SIZE = 300

ENC_1_RNN_SIZE = 300
ENC_2_RNN_SIZE = 150
DEC_RNN_SIZE = 150
DROPOUT_RATE = 0.5
RNN_DROPOUT_RATE = 0.2
NUM_EPOCHS = 512
BATCH_SIZE = 79
STEPS_PER_EPOCH = 20
TEST_STEPS = len(x_test)//BATCH_SIZE

VALIDATION_STEPS = 3

## Import Libraries

In [5]:
from keras.layers import Input, Embedding, Dropout, Bidirectional, LSTM, RepeatVector, concatenate, TimeDistributed, Dense
from keras.models import Model
import keras.backend as K
from keras.callbacks import*
from keras.utils import to_categorical

Using TensorFlow backend.


## Build Graph

In [8]:
K.clear_session()
sequence = Input(shape=(MAX_SENT_LEN,), name='INPUT') 
emb_seq = Embedding(VOCAB_SIZE, EMBEDDING_SIZE, weights=[embedding], mask_zero=True, input_length=MAX_SENT_LEN, trainable=False, name='EMBEDDING')(sequence)
tf_seq = Input(shape=(MAX_ADJL_LEN,), name='TF_INPUT')
tf_emb = Embedding(NUM_CLASSES, TF_EMBEDDING_SIZE, weights=[embedding], mask_zero=True, input_length=MAX_ADJL_LEN, trainable=False, name='TF_EMBEDDING')(tf_seq)
blstm = Bidirectional(LSTM(ENC_1_RNN_SIZE, return_sequences=True, implementation=0, dropout=RNN_DROPOUT_RATE, recurrent_dropout=RNN_DROPOUT_RATE), merge_mode='concat', name='ENC_BLSTM_1')(emb_seq)
blstm = Dropout(DROPOUT_RATE)(blstm)
blstm = Bidirectional(LSTM(ENC_2_RNN_SIZE, return_sequences=False, implementation=0, dropout=RNN_DROPOUT_RATE, recurrent_dropout=RNN_DROPOUT_RATE), merge_mode='concat', name='ENC_BLSTM_2')(blstm)
blstm = Dropout(DROPOUT_RATE)(blstm)
context = RepeatVector(MAX_ADJL_LEN, name='CONTEXT')(blstm)
context = concatenate([context, tf_emb], axis=-1)
blstm = Bidirectional(LSTM(DEC_RNN_SIZE, return_sequences=True, implementation=0, dropout=RNN_DROPOUT_RATE, recurrent_dropout=RNN_DROPOUT_RATE, name='DEC_LSTM'), merge_mode='concat', name='DEC_BLSTM')(context)
lstm = Dropout(DROPOUT_RATE)(blstm)
output = TimeDistributed(Dense(NUM_CLASSES, activation='softmax'), name='OUTPUT')(lstm)
model = Model(inputs=[sequence, tf_seq], outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

## Model Visualization

In [9]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
INPUT (InputLayer)               (None, 64)            0                                            
____________________________________________________________________________________________________
EMBEDDING (Embedding)            (None, 64, 300)       17652300    INPUT[0][0]                      
____________________________________________________________________________________________________
ENC_BLSTM_1 (Bidirectional)      (None, 64, 600)       1442400     EMBEDDING[0][0]                  
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 64, 600)       0           ENC_BLSTM_1[0][0]                
___________________________________________________________________________________________

## Train

In [10]:
def gen_label(s):
    """
    One-hot encoding
    """
    gen = to_categorical(s, num_classes=NUM_CLASSES)
    return gen

def data_generator_all(data, label, batch_size):
    """
    Yield batches of all data
    """
    count = 0
    while True:
        if count >= len(data[0]): 
            count = 0
        x_1 = np.zeros((batch_size, MAX_SENT_LEN))
        x_2 = np.zeros((batch_size, MAX_ADJL_LEN))
        y = np.zeros((batch_size, MAX_ADJL_LEN, NUM_CLASSES))
        for i in range(batch_size):
            n = i + count
            if n > len(data[0])-1:
                break
            x_1[i, :] = data[0][n]
            x_2[i, :] = data[1][n]
            y[i, :, :] = gen_label(label[n])
        count += batch_size
        yield ([x_1, x_2], y)
        
def data_generator(data, label, batch_size): 
    """
    Yield batches 
    """
    index = np.arange(len(data[0]))
    np.random.shuffle(index)    
    batches = [index[range(batch_size*i, min(len(data[0]), batch_size*(i+1)))] for i in range(len(data[0])//batch_size)]
    while True:
        for i in batches:
            x_1 = data[0][i]
            x_2 = data[1][i]
            y = np.array(list(map(gen_label, label[i])))
            yield ([x_1, x_2], y)

In [11]:
gen_train_all = data_generator([x_train_all, tf_train_all], y_train_all, BATCH_SIZE)
gen_test = data_generator_all([x_test, tf_test], y_test, BATCH_SIZE)
#gen_train = data_generator(x_train, y_train, BATCH_SIZE)
#gen_val = data_generator(x_val, y_val, BATCH_SIZE)

In [12]:
# Continue Trian
# filename = 'cp_logs/.hdf5'
# model.load_weights(filename)
# model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

In [13]:
filepath = 'cp_logs/weights.{epoch:03d}-{val_loss:.6f}.hdf5'
log_string = 'tb_logs/tf'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True)
tensorboard = TensorBoard(log_dir=log_string, 
                          histogram_freq=1, 
                          write_graph=False, 
                          write_grads=False, 
                          batch_size=BATCH_SIZE, 
                          write_images=True, 
                          embeddings_freq=1, 
                          embeddings_layer_names=None,
                          embeddings_metadata=None) 

In [12]:
history = model.fit_generator(gen_train_all, 
                              steps_per_epoch=STEPS_PER_EPOCH, 
                              epochs=NUM_EPOCHS, 
                              verbose=1,
                              callbacks=[checkpoint, tensorboard],
                              validation_data=gen_test, 
                              validation_steps=TEST_STEPS)

Epoch 1/512
20/20 [==============================] - 23s - loss: 7.7333 - val_loss: 6.5366
Epoch 2/512
20/20 [==============================] - 45s - loss: 6.1098 - val_loss: 6.1826
Epoch 3/512
20/20 [==============================] - 50s - loss: 5.7357 - val_loss: 5.8894
Epoch 4/512
20/20 [==============================] - 46s - loss: 5.4694 - val_loss: 5.7388
Epoch 5/512
20/20 [==============================] - 48s - loss: 5.3558 - val_loss: 5.7200
Epoch 6/512
20/20 [==============================] - 50s - loss: 5.2756 - val_loss: 5.6850
Epoch 7/512
20/20 [==============================] - 50s - loss: 5.1401 - val_loss: 5.7224
Epoch 8/512
20/20 [==============================] - 54s - loss: 5.1572 - val_loss: 5.6203
Epoch 9/512
20/20 [==============================] - 50s - loss: 5.1001 - val_loss: 5.4577
Epoch 10/512
20/20 [==============================] - 56s - loss: 5.0194 - val_loss: 5.2991
Epoch 11/512
20/20 [==============================] - 56s - loss: 4.8899 - val_loss: 5.28

20/20 [==============================] - 51s - loss: 2.6373 - val_loss: 3.2684
Epoch 70/512
20/20 [==============================] - 48s - loss: 2.6424 - val_loss: 3.2426
Epoch 71/512
20/20 [==============================] - 49s - loss: 2.5667 - val_loss: 3.2615
Epoch 72/512
20/20 [==============================] - 50s - loss: 2.5732 - val_loss: 3.1625
Epoch 73/512
20/20 [==============================] - 52s - loss: 2.6622 - val_loss: 3.2125
Epoch 74/512
20/20 [==============================] - 53s - loss: 2.5197 - val_loss: 3.2710
Epoch 75/512
20/20 [==============================] - 51s - loss: 2.4910 - val_loss: 3.1932
Epoch 76/512
20/20 [==============================] - 50s - loss: 2.6736 - val_loss: 3.1642
Epoch 77/512
20/20 [==============================] - 54s - loss: 2.4572 - val_loss: 3.2532
Epoch 78/512
20/20 [==============================] - 49s - loss: 2.4602 - val_loss: 3.1684
Epoch 79/512
20/20 [==============================] - 50s - loss: 2.5453 - val_loss: 3.1569
E

20/20 [==============================] - 52s - loss: 2.0079 - val_loss: 2.9010
Epoch 147/512
20/20 [==============================] - 53s - loss: 1.9935 - val_loss: 2.9254
Epoch 148/512
20/20 [==============================] - 53s - loss: 2.0415 - val_loss: 2.8907
Epoch 149/512
20/20 [==============================] - 53s - loss: 2.0486 - val_loss: 2.8402
Epoch 150/512
20/20 [==============================] - 51s - loss: 2.0434 - val_loss: 2.9127
Epoch 151/512
20/20 [==============================] - 48s - loss: 1.9768 - val_loss: 2.9055
Epoch 152/512
20/20 [==============================] - 52s - loss: 1.9415 - val_loss: 2.8336
Epoch 153/512
20/20 [==============================] - 54s - loss: 1.9564 - val_loss: 2.8105
Epoch 154/512
20/20 [==============================] - 54s - loss: 1.9073 - val_loss: 2.8018
Epoch 155/512
20/20 [==============================] - 54s - loss: 2.0464 - val_loss: 2.8619
Epoch 156/512
20/20 [==============================] - 50s - loss: 1.9084 - val_loss

20/20 [==============================] - 50s - loss: 1.8157 - val_loss: 2.6732
Epoch 227/512
20/20 [==============================] - 54s - loss: 1.7298 - val_loss: 2.6456
Epoch 228/512
20/20 [==============================] - 52s - loss: 1.8419 - val_loss: 2.6749
Epoch 229/512
20/20 [==============================] - 53s - loss: 1.7358 - val_loss: 2.7047
Epoch 230/512
20/20 [==============================] - 50s - loss: 1.7994 - val_loss: 2.6868
Epoch 231/512
20/20 [==============================] - 50s - loss: 1.7296 - val_loss: 2.7131
Epoch 232/512
20/20 [==============================] - 51s - loss: 1.8336 - val_loss: 2.6758
Epoch 233/512
20/20 [==============================] - 50s - loss: 1.7157 - val_loss: 2.7039
Epoch 234/512
20/20 [==============================] - 52s - loss: 1.7399 - val_loss: 2.7647
Epoch 235/512
20/20 [==============================] - 47s - loss: 1.7498 - val_loss: 2.6752
Epoch 236/512
20/20 [==============================] - 52s - loss: 1.7783 - val_loss

20/20 [==============================] - 53s - loss: 1.6604 - val_loss: 2.6714
Epoch 308/512
20/20 [==============================] - 54s - loss: 1.6298 - val_loss: 2.6866
Epoch 309/512
20/20 [==============================] - 50s - loss: 1.6143 - val_loss: 2.6422
Epoch 310/512
20/20 [==============================] - 53s - loss: 1.7387 - val_loss: 2.6522
Epoch 311/512
20/20 [==============================] - 50s - loss: 1.6386 - val_loss: 2.6611
Epoch 312/512
20/20 [==============================] - 53s - loss: 1.5857 - val_loss: 2.6846
Epoch 313/512
20/20 [==============================] - 47s - loss: 1.6592 - val_loss: 2.6518
Epoch 314/512
20/20 [==============================] - 49s - loss: 1.6874 - val_loss: 2.6495
Epoch 315/512
20/20 [==============================] - 53s - loss: 1.6100 - val_loss: 2.6327
Epoch 316/512
20/20 [==============================] - 54s - loss: 1.6607 - val_loss: 2.6518
Epoch 317/512
20/20 [==============================] - 51s - loss: 1.5902 - val_loss

20/20 [==============================] - 54s - loss: 1.6761 - val_loss: 2.6182
Epoch 350/512
20/20 [==============================] - 53s - loss: 1.6448 - val_loss: 2.6436
Epoch 351/512
20/20 [==============================] - 48s - loss: 1.5834 - val_loss: 2.6817
Epoch 352/512
20/20 [==============================] - 51s - loss: 1.6114 - val_loss: 2.6793
Epoch 353/512
20/20 [==============================] - 53s - loss: 1.6188 - val_loss: 2.6708
Epoch 354/512
20/20 [==============================] - 53s - loss: 1.6528 - val_loss: 2.6025
Epoch 355/512
20/20 [==============================] - 54s - loss: 1.6348 - val_loss: 2.6504
Epoch 356/512
20/20 [==============================] - 53s - loss: 1.5404 - val_loss: 2.6272
Epoch 357/512
20/20 [==============================] - 49s - loss: 1.5634 - val_loss: 2.6230
Epoch 358/512
20/20 [==============================] - 53s - loss: 1.5334 - val_loss: 2.6073
Epoch 359/512
20/20 [==============================] - 53s - loss: 1.5451 - val_loss

20/20 [==============================] - 50s - loss: 1.6463 - val_loss: 2.6741
Epoch 434/512
20/20 [==============================] - 52s - loss: 1.5621 - val_loss: 2.7166
Epoch 435/512
20/20 [==============================] - 51s - loss: 1.5797 - val_loss: 2.6679
Epoch 436/512
20/20 [==============================] - 54s - loss: 1.5803 - val_loss: 2.6914
Epoch 437/512
20/20 [==============================] - 53s - loss: 1.6687 - val_loss: 2.6452
Epoch 438/512
20/20 [==============================] - 51s - loss: 1.6003 - val_loss: 2.6929
Epoch 439/512
20/20 [==============================] - 53s - loss: 1.5896 - val_loss: 2.7183
Epoch 440/512
20/20 [==============================] - 53s - loss: 1.5993 - val_loss: 2.7054
Epoch 441/512
20/20 [==============================] - 53s - loss: 1.6379 - val_loss: 2.6711
Epoch 442/512
20/20 [==============================] - 50s - loss: 1.6386 - val_loss: 2.6810
Epoch 443/512
20/20 [==============================] - 50s - loss: 1.6136 - val_loss

# Evaluate

## Naive Evaluate

In [19]:
filename = 'cp_logs/weights.336-2.599974.hdf5'
model.load_weights(filename)

result = np.argmax(model.predict([x_test, tf_test], batch_size=BATCH_SIZE, verbose=1), -1)

395/395 [==============================] - 1s     


In [14]:
preResult = [[index2word[i] for i in sent if i != 0] for sent in result]
actResult = [[index2word[i] for i in sent if i != 0] for sent in y_test]
e1Count = sum([1 for i in range(395) if actResult[i][0] == preResult[i][0]])
e2Count = sum([1 for i in range(395) if actResult[i][1] == preResult[i][1]])
enCount = sum([1 for i in range(395) if actResult[i][0] == preResult[i][0] and actResult[i][1] == preResult[i][1]])
tripleCount = sum([1 for i in range(395) if actResult[i][0] == preResult[i][0] and actResult[i][1] == preResult[i][1] and actResult[i][2] == preResult[i][2]])
print('E1 Accuracy: \t\t%.6f' % (e1Count/395))
print('E2 Accuracy: \t\t%.6f' % (e2Count/395))
print('En Accuracy: \t\t%.6f' % (enCount/395))
print('Triple Accuracy: \t%.6f' % (tripleCount/395))

E1 Accuracy: 		0.600000
E2 Accuracy: 		0.653165
En Accuracy: 		0.362025
Triple Accuracy: 	0.278481


In [15]:
for i in range(395):
    print('Predict: \t%s' % ' '.join(preResult[i]))
    print('Ground-Truth: \t%s' % ' '.join(actResult[i]))
    print('---')

Predict: 	europe italy /location/country/administrative_divisions
Ground-Truth: 	spain italy /location/administrative_division/country
---
Predict: 	westchester connecticut /location/location/contains
Ground-Truth: 	westchester connecticut /location/administrative_division/country
---
Predict: 	ariel israel /people/person/nationality
Ground-Truth: 	dalia israel /people/person/nationality
---
Predict: 	bangalore bihar /location/location/contains
Ground-Truth: 	hyderabad potti /location/location/contains
---
Predict: 	elizabeth north /people/person/place_lived
Ground-Truth: 	kent north /people/person/place_lived
---
Predict: 	ontario waterloo /location/location/contains
Ground-Truth: 	ontario waterloo /location/location/contains
---
Predict: 	baltimore american /business/company/founders
Ground-Truth: 	blackstone stephen /business/company/founders
---
Predict: 	europe italy /location/country/administrative_divisions
Ground-Truth: 	france italy /location/administrative_division/country
--

## Greedy 1-best Search Evaluate

In [68]:
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm

In [69]:
filename = 'cp_logs/weights.336-2.599974.hdf5'
model.load_weights(filename)

In [71]:
preResult = []
for i in tqdm(range(395)):
    pR = []
    tag = 0
    fb_words = np.array([58840])
    x = np.array([x_test[i]])
    
    for j in range(MAX_ADJL_LEN):
        fb_input = pad_sequences([fb_words], maxlen=MAX_ADJL_LEN, padding='post', truncating='post')
        prob = model.predict([x, fb_input], batch_size=BATCH_SIZE)[0][j]
        if j < 2:
            for item in [i for i in x_test[i] if i != 0]:
                prob[item] = float(prob[item]+1)
        if j == 2:
            prob[:58828] = -1
        prob[0] = -1
        prob[tag] = -1
        result = prob.argmax()
        tag = result
        pR.append(result)
        fb_words = np.append(fb_words, result)
    preResult.append(pR)


100%|██████████| 395/395 [02:18<00:00,  2.93it/s]


In [82]:
preResult = [[index2word[i] for i in sent if i != 0] for sent in preResult]
actResult = [[index2word[i] for i in sent if i != 0] for sent in y_test]
e1Count = sum([1 for i in range(395) if actResult[i][0] == preResult[i][0]])
e2Count = sum([1 for i in range(395) if actResult[i][1] == preResult[i][1]])
enCount = sum([1 for i in range(395) if actResult[i][0] == preResult[i][0] and actResult[i][1] == preResult[i][1]])
tripleCount = sum([1 for i in range(395) if actResult[i][0] == preResult[i][0] and actResult[i][1] == preResult[i][1] and actResult[i][2] == preResult[i][2]])
print('E1 Accuracy: \t\t%.6f' % (e1Count/395))
print('E2 Accuracy: \t\t%.6f' % (e2Count/395))
print('En Accuracy: \t\t%.6f' % (enCount/395))
print('Triple Accuracy: \t%.6f' % (tripleCount/395))

E1 Accuracy: 		0.291139
E2 Accuracy: 		0.281013
En Accuracy: 		0.139241
Triple Accuracy: 	0.113924


In [84]:
for i in range(395):
    print('Predict: \t%s' % ' '.join(preResult[i]))
    print('Ground-Truth: \t%s' % ' '.join(actResult[i]))
    print('---')

Predict: 	mexico spain /location/country/administrative_divisions
Ground-Truth: 	spain italy /location/administrative_division/country
---
Predict: 	westchester long /location/location/contains
Ground-Truth: 	westchester connecticut /location/administrative_division/country
---
Predict: 	israel saudi /location/location/contains
Ground-Truth: 	dalia israel /people/person/nationality
---
Predict: 	india university /location/location/contains
Ground-Truth: 	hyderabad potti /location/location/contains
---
Predict: 	north bush /location/location/contains
Ground-Truth: 	kent north /people/person/place_lived
---
Predict: 	ontario waterloo /location/location/contains
Ground-Truth: 	ontario waterloo /location/location/contains
---
Predict: 	stephen a. /business/person/company
Ground-Truth: 	blackstone stephen /business/company/founders
---
Predict: 	mexico spain /location/country/administrative_divisions
Ground-Truth: 	france italy /location/administrative_division/country
---
Predict: 	iran ru

## Beam Search Evaluate

In [88]:
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm

In [89]:
filename = 'cp_logs/weights.336-2.599974.hdf5'
model.load_weights(filename)

In [126]:
def beamsearch(NUM_BEAMS):
    preResult = []
    for i in tqdm(range(395)):
        fb_words = np.array([58840])
        x = np.array([x_test[i]])
    
        st_words = np.array([58840])
        st_input = pad_sequences([st_words], maxlen=MAX_ADJL_LEN, padding='post', truncating='post')
        prob = model.predict([x, st_input], batch_size=BATCH_SIZE)[0][0]
        prob[0] = -1
        for item in [i for i in x_test[i] if i != 0]:
            prob[item] = float(prob[item]+1)
        beam = {}
        for _ in range(NUM_BEAMS):
            b = prob.argmax()
            beam[b] = float(prob[b])
            prob[b] = -1
        
        allResult = []
        for b, s in beam.items():
            pR = [b]
            fb_words = np.append(fb_words, b)
            tag = b
            for j in range(1, MAX_ADJL_LEN):
                fb_input = pad_sequences([fb_words], maxlen=MAX_ADJL_LEN, padding='post', truncating='post')
                prob = model.predict([x, fb_input], batch_size=BATCH_SIZE)[0][j]
                if j == 1:
                    for item in [i for i in x_test[i] if i != 0]:
                        prob[item] = float(prob[item]+1)
                if j == 2:
                    prob[:58828] = -1
                prob[0] = -1
                prob[tag] = -1
                result = prob.argmax()
                beam[b] += float(prob[result])
                tag = result
                pR.append(result)
                fb_words = np.append(fb_words, result)            
            allResult.append(pR)
        pR = sum([i for i in allResult if i[0] == max(beam, key=lambda x: beam[x])], [])
        preResult.append(pR)

    preResult = [[index2word[i] for i in sent if i != 0] for sent in preResult]
    actResult = [[index2word[i] for i in sent if i != 0] for sent in y_test]
    e1Count = sum([1 for i in range(395) if actResult[i][0] == preResult[i][0]])
    e2Count = sum([1 for i in range(395) if actResult[i][1] == preResult[i][1]])
    enCount = sum([1 for i in range(395) if actResult[i][0] == preResult[i][0] and actResult[i][1] == preResult[i][1]])
    reCount = sum([1 for i in range(395) if actResult[i][2] == preResult[i][2]])
    tripleCount = sum([1 for i in range(395) if actResult[i][0] == preResult[i][0] and actResult[i][1] == preResult[i][1] and actResult[i][2] == preResult[i][2]])
    print('Beam: \t\t\t%d' % NUM_BEAMS)
    print('E1 Accuracy: \t\t%.6f' % (e1Count/395))
    print('E2 Accuracy: \t\t%.6f' % (e2Count/395))
    print('En Accuracy: \t\t%.6f' % (enCount/395))
    print('Re Accuracy: \t\t%.6f' % (reCount/395))
    print('Triple Accuracy: \t%.6f' % (tripleCount/395))
    print('--------------------------------')

In [128]:
for NUM_BEAMS in range(1, 3):
    beamsearch(NUM_BEAMS)

  0%|          | 0/395 [00:00<?, ?it/s]

Beam: 			1
E1 Accuracy: 		0.291139
E2 Accuracy: 		0.281013
En Accuracy: 		0.139241
Re Accuracy: 		0.506329
Triple Accuracy: 	0.113924
--------------------------------


100%|██████████| 395/395 [03:46<00:00,  1.36it/s]

Beam: 			2
E1 Accuracy: 		0.278481
E2 Accuracy: 		0.298734
En Accuracy: 		0.113924
Re Accuracy: 		0.506329
Triple Accuracy: 	0.075949
--------------------------------


In [ ]:
for i in range(395):
    print('Predict: \t%s' % ' '.join(preResult[i]))
    print('Ground-Truth: \t%s' % ' '.join(actResult[i]))
    print('---')